In [1]:
# Create the directory if it doesn't exist
!mkdir -p ../data

# Download files using curl
!curl -o ../data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o ../data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1330k  100 1330k    0     0  6442k      0 --:--:-- --:--:-- --:--:-- 6458k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1253k  100 1253k    0     0  6779k      0 --:--:-- --:--:-- --:--:-- 6813k


In [1]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [2]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [3]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [4]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [5]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [6]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [7]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Accessing as dafnetamayo

Initialized MLflow to track repo "dafnetamayo/nyc-taxi-time-prediction"

Repository dafnetamayo/nyc-taxi-time-prediction initialized!

https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow


<Experiment: artifact_location='mlflow-artifacts:/5b5314a67f894d4d9aa944625869aa1c', creation_time=1726630094011, experiment_id='0', last_update_time=1726630094011, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

In [9]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("dafnetamayo/nyc-taxi-time-prediction")

# Upload file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-01.parquet",  # local path of file to upload
    Key="train_data.parquet",  # remote path where to upload the file
)
# Download file
# s3.download_file(
#     Bucket="nyc-taxi-time-prediction",  # name of the repo
#     Key="remote.csv",  #  remote path from where to download the file
#     Filename="local.csv",  # local path where to download the file
# )

Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

In [10]:
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-01.parquet",  # local path of file to upload
    Key="eval_data.parquet",  # remote path where to upload the file
)

In [15]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib


In [16]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [17]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = root_mean_squared_error(y_val, y_pred)
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [18]:
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

[0]	validation-rmse:5.68376                           
[1]	validation-rmse:5.35654                           
[2]	validation-rmse:5.30899                           
[3]	validation-rmse:5.29479                           
[4]	validation-rmse:5.28483                           
[5]	validation-rmse:5.27810                           
[6]	validation-rmse:5.27204                           
[7]	validation-rmse:5.25502                           
[8]	validation-rmse:5.25180                           
[9]	validation-rmse:5.25192                           
[10]	validation-rmse:5.25092                          
[11]	validation-rmse:5.24974                          
[12]	validation-rmse:5.24704                          
[13]	validation-rmse:5.24414                          
[14]	validation-rmse:5.24459                          
[15]	validation-rmse:5.24276                          
[16]	validation-rmse:5.23632                          
[17]	validation-rmse:5.23424                          
[18]	valid

2024/09/17 21:44:20 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:44:20 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/daftamayo/Documents/DagsHub/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:44:20] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:44:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

/Users/daftamayo/Documents/DagsHub/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:44:24] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:137

[0]	validation-rmse:7.39105                                                    
[1]	validation-rmse:6.42938                                                    
[2]	validation-rmse:5.91242                                                    
[3]	validation-rmse:5.63964                                                    
[4]	validation-rmse:5.50273                                                    
[5]	validation-rmse:5.41373                                                    
[6]	validation-rmse:5.37508                                                    
[7]	validation-rmse:5.35598                                                    
[8]	validation-rmse:5.34435                                                    
[9]	validation-rmse:5.33431                                                    
[10]	validation-rmse:5.32816                                                   
[11]	validation-rmse:5.32046                                                   
[12]	validation-rmse:5.31437            

2024/09/17 21:44:39 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:44:39 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/daftamayo/Documents/DagsHub/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:44:39] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:44:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

/Users/daftamayo/Documents/DagsHub/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:44:42] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:137

[0]	validation-rmse:8.65226                                                    
[1]	validation-rmse:8.24384                                                    
[2]	validation-rmse:7.88157                                                    
[3]	validation-rmse:7.56101                                                    
[4]	validation-rmse:7.27848                                                    
[5]	validation-rmse:7.03018                                                    
[6]	validation-rmse:6.81278                                                    
[7]	validation-rmse:6.62280                                                    
[8]	validation-rmse:6.45711                                                    
[9]	validation-rmse:6.31328                                                    
[10]	validation-rmse:6.18869                                                   
[11]	validation-rmse:6.08067                                                   
[12]	validation-rmse:5.98746            

...

2024/09/17 21:46:27 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:46:27 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/daftamayo/Documents/DagsHub/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:46:27] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:46:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

/Users/daftamayo/Documents/DagsHub/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:46:30] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:137

[0]	validation-rmse:5.68905                                                    
[1]	validation-rmse:5.46011                                                    
[2]	validation-rmse:5.43510                                                    
[3]	validation-rmse:5.42470                                                    
[4]	validation-rmse:5.42618                                                    
[5]	validation-rmse:5.42609                                                    
[6]	validation-rmse:5.40019                                                    
[7]	validation-rmse:5.39528                                                    
[8]	validation-rmse:5.39562                                                    
[9]	validation-rmse:5.38741                                                    
[10]	validation-rmse:5.38080                                                   
[11]	validation-rmse:5.37914                                                   
[12]	validation-rmse:5.37341            

2024/09/17 21:46:42 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:46:42 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/daftamayo/Documents/DagsHub/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:46:42] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:46:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

/Users/daftamayo/Documents/DagsHub/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:46:46] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:137

100%|██████████| 10/10 [02:40<00:00, 16.04s/trial, best loss: 5.204264278279571]
[0]	validation-rmse:5.64677
[1]	validation-rmse:5.32879
[2]	validation-rmse:5.29135
[3]	validation-rmse:5.26789
[4]	validation-rmse:5.25967
[5]	validation-rmse:5.25804
[6]	validation-rmse:5.25619
[7]	validation-rmse:5.24239
[8]	validation-rmse:5.23906
[9]	validation-rmse:5.23150
[10]	validation-rmse:5.22958
[11]	validation-rmse:5.22852
[12]	validation-rmse:5.21941
[13]	validation-rmse:5.21823
[14]	validation-rmse:5.21136
[15]	validation-rmse:5.21083
[16]	validation-rmse:5.20857
[17]	validation-rmse:5.20839
[18]	validation-rmse:5.20765
[19]	validation-rmse:5.20638
[20]	validation-rmse:5.20565
[21]	validation-rmse:5.20625
[22]	validation-rmse:5.20608
[23]	validation-rmse:5.20722
[24]	validation-rmse:5.20564
[25]	validation-rmse:5.20471
[26]	validation-rmse:5.20282
[27]	validation-rmse:5.20348
[28]	validation-rmse:5.20354
[29]	validation-rmse:5.20364
[30]	validation-rmse:5.20227
[31]	validation-rmse:5.20405
[

2024/09/17 21:46:57 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/09/17 21:46:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/daftamayo/Documents/DagsHub/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:46:57] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/09/17 21:46:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/17 21:47:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run Xgboost Hyper-parameter Optimization at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#

In [19]:
best_params

{'learning_rate': np.float64(0.7513192986209228),
 'max_depth': 49,
 'min_child_weight': np.float64(4.197733996818088),
 'reg_alpha': np.float64(0.009008469281388612),
 'reg_lambda': np.float64(0.013446719669052455),
 'seed': 42,
 'objective': 'reg:squarederror'}

In [20]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Successfully registered model 'nyc-taxi-model'.
2024/09/17 21:48:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 1
Created version '1' of model 'nyc-taxi-model'.


#b37fae6bbfd64f0cafdf628b03660103

In [21]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Model registry for the NYC Taxi Time Prediction Project",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1726631335643, current_stage='None', description='The model version 1 was transitioned to champion on 2024-09-17 21:49:57.062626', last_updated_timestamp=1726631397538, name='nyc-taxi-model', run_id='b37fae6bbfd64f0cafdf628b03660103', run_link='', source='mlflow-artifacts:/5b5314a67f894d4d9aa944625869aa1c/b37fae6bbfd64f0cafdf628b03660103/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [22]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

array([19.099905, 28.762825,  7.438208, ..., 32.80175 , 13.659024,
       20.112152], dtype=float32)